In [2]:
# Install dependencies (watch out for versions)
!pip install datasets==2.8.0
!pip install transformers==4.26.0
!pip install huggingface-hub==0.13.0
!pip install rouge_score==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.3
    Uninstalling huggingface-hub-0.19.3:
      Successfully uninstalled huggingface-hub-0.19.3
  Prepa

In [8]:
# Download T5-large library
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-large")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
# If using Colab, run this to select CSVs to upload
from google.colab import files
uploaded = files.upload()

Saving treatment_sequence_data.csv to treatment_sequence_data (1).csv


In [10]:
# Read CSVs with training data (will use only 1 or the other, depending on type of recommendation system)
# Each row represents a list of sports equipment one has purchased (input), and the next item to recommend (output)
# Change file path as needed
import io
import pandas as pd
df_english_training = pd.read_csv('treatment_sequence_data.csv')
# Dataset is now stored in a Pandas Dataframe

In [11]:
df_english_training

,Input,Output
0,"invalid, methamphetamine",cocaine/crack
1,"alcohol, marijuana/hashish, cocaine/crack, met...",invalid
2,"none, invalid, cocaine/crack, heroin",invalid
3,"none, invalid, alcohol, marijuana/hashish",cocaine/crack
4,"none, invalid, alcohol",cocaine/crack
...,...,...
97,"none, invalid, methamphetamine",cocaine/crack
98,"none, other stimulants, marijuana/hashish, met...",invalid
99,"alcohol, marijuana/hashish, none, invalid, coc...",invalid
100,"alcohol, marijuana/hashish, methamphetamine",invalid


In [22]:
# Full lists of treatments available

items_english_list = ['other stimulants', 'hypnotics', 'alcohol', 'invalid', 'benzodiazepines', 'inhalants', 'none', 'other opiates/synthetics', 'non-prescription methadone',
                       'non-rx methadone', 'other amphetamines', 'other opiates and synthetics', 'marijuana/hashish', 'other hallucinogens', 'heroin',
                       'methamphetamine', 'cocaine/crack']


print(len(items_english_list))
print(items_english_list)

17
['other stimulants', 'hypnotics', 'alcohol', 'invalid', 'benzodiazepines', 'inhalants', 'none', 'other opiates/synthetics', 'non-prescription methadone', 'non-rx methadone', 'other amphetamines', 'other opiates and synthetics', 'marijuana/hashish', 'other hallucinogens', 'heroin', 'methamphetamine', 'cocaine/crack']


In [23]:
df_english_training['Input'].iloc[0]

'invalid, methamphetamine'

In [28]:
# Functions to select the list of products (sports equipment) not yet purchased for the given customer...
# Will use these as candidates for next recommended item

def get_items_not_applied_yet(history):
  items_not_applied_yet = []

  for item in items_english_list:
    if(item not in history): #If the item (from the list) is NOT in the history string
      items_not_applied_yet.append(item)
  return items_not_applied_yet


In [18]:
# Functions that will build strings for list of candidates for recommendation and list of items purchased

def modify_undone_items_list_english(Unpurchased_items_list):
  concatenated_list = ', '.join(Unpurchased_items_list) #Join into a string with commas in between
  concatenated_list = concatenated_list.rstrip(',') #Drop last comma
  concatenated_list = "CANDIDATES FOR RECOMMENDATION: {" + concatenated_list + "}"
  return concatenated_list


def modify_applied_items_english(purchase_history):
  purchase_history = "TREATMENT COMPLETED: {" + purchase_history + "}"
  return purchase_history


In [29]:
# Add columns with strings of items not yet purchased by the customer

df_english_training['Undone_treatment'] = df_english_training['Input'].apply(get_items_not_applied_yet)



In [25]:
len(df_english_training['Undone_treatment'].iloc[0])

15

In [26]:
print(df_english_training['Undone_treatment'].iloc[0])

['other stimulants', 'hypnotics', 'alcohol', 'benzodiazepines', 'inhalants', 'none', 'other opiates/synthetics', 'non-prescription methadone', 'non-rx methadone', 'other amphetamines', 'other opiates and synthetics', 'marijuana/hashish', 'other hallucinogens', 'heroin', 'cocaine/crack']


In [27]:
print(df_english_training['Input'].iloc[0])

invalid, methamphetamine


In [30]:
df_english_training['Input'].apply(modify_applied_items_english).iloc[0]

'TREATMENT COMPLETED: {invalid, methamphetamine}'

In [31]:
df_english_training['Undone_treatment'].apply(modify_undone_items_list_english).iloc[0]

'CANDIDATES FOR RECOMMENDATION: {other stimulants, hypnotics, alcohol, benzodiazepines, inhalants, none, other opiates/synthetics, non-prescription methadone, non-rx methadone, other amphetamines, other opiates and synthetics, marijuana/hashish, other hallucinogens, heroin, cocaine/crack}'

In [32]:
# Combine purchased items and unpurchased items strings to build prompt
df_english_training['Prompt'] = df_english_training['Input'].apply(modify_applied_items_english) + \
                               " - " + df_english_training['Undone_treatment'].apply(modify_undone_items_list_english) + \
                               " - RECOMMENDATION: "

In [33]:
df_english_training

,Input,Output,Undone_treatment,Prompt
0,"invalid, methamphetamine",cocaine/crack,"[other stimulants, hypnotics, alcohol, benzodi...","TREATMENT COMPLETED: {invalid, methamphetamine..."
1,"alcohol, marijuana/hashish, cocaine/crack, met...",invalid,"[other stimulants, hypnotics, invalid, benzodi...","TREATMENT COMPLETED: {alcohol, marijuana/hashi..."
2,"none, invalid, cocaine/crack, heroin",invalid,"[other stimulants, hypnotics, alcohol, benzodi...","TREATMENT COMPLETED: {none, invalid, cocaine/c..."
3,"none, invalid, alcohol, marijuana/hashish",cocaine/crack,"[other stimulants, hypnotics, benzodiazepines,...","TREATMENT COMPLETED: {none, invalid, alcohol, ..."
4,"none, invalid, alcohol",cocaine/crack,"[other stimulants, hypnotics, benzodiazepines,...","TREATMENT COMPLETED: {none, invalid, alcohol} ..."
...,...,...,...,...
97,"none, invalid, methamphetamine",cocaine/crack,"[other stimulants, hypnotics, alcohol, benzodi...","TREATMENT COMPLETED: {none, invalid, methamphe..."
98,"none, other stimulants, marijuana/hashish, met...",invalid,"[hypnotics, alcohol, invalid, benzodiazepines,...","TREATMENT COMPLETED: {none, other stimulants, ..."
99,"alcohol, marijuana/hashish, none, invalid, coc...",invalid,"[other stimulants, hypnotics, benzodiazepines,...","TREATMENT COMPLETED: {alcohol, marijuana/hashi..."
100,"alcohol, marijuana/hashish, methamphetamine",invalid,"[other stimulants, hypnotics, invalid, benzodi...","TREATMENT COMPLETED: {alcohol, marijuana/hashi..."


In [34]:
df_english_training['Prompt'].iloc[0]

'TREATMENT COMPLETED: {invalid, methamphetamine} - CANDIDATES FOR RECOMMENDATION: {other stimulants, hypnotics, alcohol, benzodiazepines, inhalants, none, other opiates/synthetics, non-prescription methadone, non-rx methadone, other amphetamines, other opiates and synthetics, marijuana/hashish, other hallucinogens, heroin, cocaine/crack} - RECOMMENDATION: '

In [35]:
# Reformat training and eval dataframes
train_english_reformatted = df_english_training[['Prompt', 'Output']][0:100].reset_index().rename(columns={"Prompt":"source", "Output": "target", "index": "id"})
train_english_reformatted = train_english_reformatted.dropna()
train_english_reformatted

eval_english_reformatted = df_english_training[['Prompt', 'Output']][100:].reset_index().rename(columns={"Prompt":"source", "Output": "target", "index": "id"})
eval_english_reformatted = eval_english_reformatted.dropna()
eval_english_reformatted

,id,source,target
0,100,"TREATMENT COMPLETED: {alcohol, marijuana/hashi...",invalid
1,101,"TREATMENT COMPLETED: {none, invalid, alcohol, ...",cocaine/crack


In [36]:
# Convert dataframes to Dataset objects (for use in Hugging Face model)
from datasets import Dataset

english_dataset_train = Dataset.from_pandas(train_english_reformatted)
english_dataset_eval = Dataset.from_pandas(eval_english_reformatted)

In [37]:
# Build DatasetDict from Dataset objects
import datasets

english_data_dict_dataset = datasets.DatasetDict({"train": english_dataset_train, "eval": english_dataset_eval})
english_data_dict_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'target'],
        num_rows: 100
    })
    eval: Dataset({
        features: ['id', 'source', 'target'],
        num_rows: 2
    })
})

In [38]:
# Preprocess function to tokenize input text (for fine-tuning)

max_input_length = tokenizer.model_max_length
max_target_length = 20 # Adjust as needed, should be relatively short as we expect 1 product to be recommended

def preprocess_function(examples):
  inputs = [doc for doc in examples["source"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

  #Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["target"], max_length=max_target_length, truncation=True, padding=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [39]:
# Test preprocessing on first 2 rows
preprocess_function(english_data_dict_dataset["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[11466, 21613, 11810, 23666, 3765, 11430, 10, 3, 2, 77, 27769, 6, 140, 189, 4624, 88, 17, 8721, 2, 3, 18, 205, 9853, 26483, 21254, 5652, 4083, 6657, 329, 14920, 8015, 10, 3, 2, 9269, 12682, 2366, 6, 3, 13397, 29, 9798, 7, 6, 4798, 6, 3, 19738, 23193, 776, 102, 4477, 6, 16, 3828, 2366, 6, 5839, 6, 119, 3, 18916, 1422, 87, 7, 63, 29, 17194, 7, 6, 529, 18, 2026, 11830, 1736, 8399, 782, 6, 529, 18, 52, 226, 1736, 8399, 782, 6, 119, 11483, 88, 17, 8721, 7, 6, 119, 3, 18916, 1422, 11, 13699, 7, 6, 10434, 87, 107, 3198, 1273, 6, 119, 6358, 6809, 29, 5255, 7, 6, 22658, 6, 29309, 87, 75, 16729, 2, 3, 18, 4083, 6657, 329, 14920, 8015, 10, 1, 0], [11466, 21613, 11810, 23666, 3765, 11430, 10, 3, 2, 138, 509, 2831, 6, 10434, 87, 107, 3198, 1273, 6, 29309, 87, 75, 16729, 6, 140, 189, 4624, 88, 17, 8721, 2, 3, 18, 205, 9853, 26483, 21254, 5652, 4083, 6657, 329, 14920, 8015, 10, 3, 2, 9269, 12682, 2366, 6, 3, 13397, 29, 9798, 7, 6, 17070, 6, 3, 19738, 23193, 776, 102, 4477, 6, 16, 3828,

In [40]:
print(len(preprocess_function(english_data_dict_dataset["train"][:2])['input_ids'][0] ))
print(len(preprocess_function(english_data_dict_dataset["train"][:2])['attention_mask'][0] ))
print(len(preprocess_function(english_data_dict_dataset["train"][:2])['labels'][0] ))

131
131
5


In [41]:
# Tokenize train and eval datasets
tokenized_datasets = english_data_dict_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    eval: Dataset({
        features: ['id', 'source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [43]:
# Instantiate Data Collator object
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [44]:
# Instantiate Data Loader for train and eval sets
# Adjust batch sizes as necessary

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["eval"], batch_size=8, collate_fn=data_collator
)

In [45]:
len(train_dataloader)

13

## Fine-tune T5 model

In [46]:
### Select Optimizer (for regularization)

from transformers import AdamW, get_scheduler

learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


65


In [47]:
# Run if you want to push to Hugging Face Hub (need account and API token)
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [48]:
# Instantiate training arguments object
batch_size = 4
args = Seq2SeqTrainingArguments(
    "./t5_recommendation_treatment",
    push_to_hub=True, # Comment out if you don't want to push to Hugging Face Hub
    evaluation_strategy = "epoch",
    learning_rate = 1e-4,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10, # Try 5-10 epochs; results may vary
    predict_with_generate = True,
    gradient_accumulation_steps = 4,
    eval_accumulation_steps = 4,
)

In [49]:
# Instantiate ROUGE metric object

from datasets import load_dataset, load_metric

metric = load_metric("rouge")
metric

<ipython-input-49-17f1ffea17c9>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [50]:
# Functions for further preprocessing and metrics computation
import numpy as np

def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [[label.strip()] for label in labels]

  return preds, labels

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  if isinstance(preds, tuple):
    preds = preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100 in the labes as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post processing
  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  result = metric.compute(predictions = decoded_preds, references = decoded_labels)
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  result["gen_len"] = np.mean(prediction_lens)

  return result

In [51]:
# Instantiate Trainer object (for fine-tuning)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["eval"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
    optimizers = (optimizer, lr_scheduler)
)

Cloning https://huggingface.co/bgalitsky/t5_recommendation_treatment into local empty directory.


In [52]:
# Train time should take a few minutes or less if on GPU
# Can take up to several hours if on CPU
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, source, id. If target, source, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 100
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 60
  Number of trainable parameters = 737668096
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,5.415375,0.000000,0.000000,0.000000,0.000000,19.000000
1,No log,2.641017,50.000000,50.000000,50.000000,50.000000,5.000000
2,No log,0.725739,50.000000,0.000000,50.000000,50.000000,2.000000
3,No log,0.436794,50.000000,50.000000,50.000000,50.000000,5.000000
4,No log,0.043078,100.000000,50.000000,100.000000,100.000000,3.500000
5,No log,0.048121,100.000000,50.000000,100.000000,100.000000,3.500000
6,No log,0.008783,100.000000,50.000000,100.000000,100.000000,3.500000
7,No log,0.052855,100.000000,50.000000,100.000000,100.000000,3.500000
8,No log,0.017095,100.000000,50.000000,100.000000,100.000000,3.500000
9,No log,0.016423,100.000000,50.000000,100.000000,100.000000,3.500000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, source, id. If target, source, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, source, id. If target, source, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

The following 

TrainOutput(global_step=60, training_loss=2.1115553538004557, metrics={'train_runtime': 128.3033, 'train_samples_per_second': 7.794, 'train_steps_per_second': 0.468, 'total_flos': 555943550976000.0, 'train_loss': 2.1115553538004557, 'epoch': 9.96})

## Collect evaluation data predictions

In [53]:
%%time
# Try predictions on validation set for confirmation
predictions = trainer.predict(tokenized_datasets["eval"])

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target, source, id. If target, source, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



CPU times: user 469 ms, sys: 34.3 ms, total: 503 ms
Wall time: 500 ms


In [54]:
predictions

PredictionOutput(predictions=array([[    0, 17070,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0, 29309,    87,    75, 16729,     1,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]]), label_ids=array([[17070,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [29309,    87,    75, 16729,     1,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]]), metrics={'test_loss': 0.01642277091741562, 'test_rouge1': 100.0, 'test_rouge2': 50.0, 'test_rougeL': 100.0, 'test_rougeLsum': 100.0, 'test_gen_len': 3.5, 'test_runtime': 0.4937, 'test_samples_per_second': 4.051, 'test_steps_per_second': 2.026})

In [55]:
# Convert tokens from data to text
def translate(tokens):
  my_list = tokenizer.convert_ids_to_tokens(tokens)
  new_list = [token for token in my_list if ('<' not in token)] # token != '<pad>' and token != '<s>'
  new_string = ''.join(new_list)
  new_string = new_string.replace("▁", " ")
  # new_string = new_string.replace("_", " ")
  new_string = new_string.strip()
  return new_string

In [60]:
len(tokenized_datasets)

2

In [61]:
# Print sample predicted output
index = 1
print(tokenized_datasets["eval"]["source"][index])
print("Target treatment outcome: ", tokenized_datasets["eval"]["target"][index])
print("Recommended treatment: ", [translate(predictions.predictions[index])])

TREATMENT COMPLETED: {none, invalid, alcohol, marijuana/hashish} - CANDIDATES FOR RECOMMENDATION: {other stimulants, hypnotics, benzodiazepines, inhalants, other opiates/synthetics, non-prescription methadone, non-rx methadone, other amphetamines, other opiates and synthetics, other hallucinogens, heroin, methamphetamine, cocaine/crack} - RECOMMENDATION: 
Target treatment outcome:  cocaine/crack
Recommended treatment:  ['cocaine/crack']


In [62]:
# Collect generated outputs and join with prompts and targets
model_generated = []
prompt_list = []
target_list = []

for i in range(len(predictions.predictions)):
  model_generated.append(translate(predictions.predictions[i]))

  prompt_list.append(english_dataset_eval['source'][i])
  target_list.append(english_dataset_eval['target'][i])

In [63]:
df_target_and_generated = pd.DataFrame()

df_target_and_generated['input'] = prompt_list
df_target_and_generated['target'] = target_list
df_target_and_generated['model_generated'] = model_generated

In [64]:
df_target_and_generated

,input,target,model_generated
0,"TREATMENT COMPLETED: {alcohol, marijuana/hashi...",invalid,invalid
1,"TREATMENT COMPLETED: {none, invalid, alcohol, ...",cocaine/crack,cocaine/crack


## Push fine-tuned model to Hugging Face Hub (optional)

In [65]:
trainer.push_to_hub("t5_recommendation_treatment")

Saving model checkpoint to ./t5_recommendation_treatment
Configuration saved in ./t5_recommendation_treatment/config.json
Configuration saved in ./t5_recommendation_treatment/generation_config.json
Model weights saved in ./t5_recommendation_treatment/pytorch_model.bin
tokenizer config file saved in ./t5_recommendation_treatment/tokenizer_config.json
Special tokens file saved in ./t5_recommendation_treatment/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 1.00/2.75G [00:00<?, ?B/s]

Upload file runs/Nov16_11-39-59_33aa629ca194/1700134837.6973076/events.out.tfevents.1700134837.33aa629ca194.59…

Upload file training_args.bin:   0%|          | 1.00/3.99k [00:00<?, ?B/s]

Upload file runs/Nov16_11-39-59_33aa629ca194/events.out.tfevents.1700134837.33aa629ca194.595.0:   0%|         …

To https://huggingface.co/bgalitsky/t5_recommendation_treatment
   f5063b4..cdd200e  main -> main

   f5063b4..cdd200e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 100.0}]}
To https://huggingface.co/bgalitsky/t5_recommendation_treatment
   cdd200e..7a32b16  main -> main

   cdd200e..7a32b16  main -> main



'https://huggingface.co/bgalitsky/t5_recommendation_treatment/commit/cdd200e3ca1b59acf96752380b03c42e5e495229'

## Test the Model on various examples!

In [3]:
# Load fine-tuned model from Hugging Face Hub
from transformers import pipeline

t5_recommender = pipeline(model="bgalitsky/t5_recommendation_treatment")

RuntimeError: ignored

In [ ]:
# Prepare sample customers (items purchased so far)
example_patient_1 = "ITEMS APPLIED: {alcohol} - CANDIDATES FOR RECOMMENDATION: {heroin, cocaine/crack, marijuana/hashish} - RECOMMENDATION: "
example_patient_2 = "ITEMS APPLIED: {marijuana/hashish, methamphetamine} - CANDIDATES FOR RECOMMENDATION: {heroin, cocaine/crack, marijuana/hashish} - RECOMMENDATION: "
example_patient_3 = "ITEMS APPLIED: { marijuana/hashish, invalid, cocaine/crack} - CANDIDATES FOR RECOMMENDATION: {heroin, cocaine/crack, marijuana/hashish} - RECOMMENDATION: "

In [ ]:
# Generate model recommendations for products for each customer
model_output_1 = t5_recommender.predict(example_patient_1)
model_recommendation_1 = model_output_1[0]['generated_text']

model_output_2 = t5_recommender.predict(example_patient_2)
model_recommendation_2 = model_output_2[0]['generated_text']

model_output_3 = t5_recommender.predict(example_patient_3)
model_recommendation_3 = model_output_3[0]['generated_text']

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



In [ ]:
print(example_patient_1)
print("RECOMMENDATION: ", model_recommendation_1)

ITEMS PURCHASED: {Soccer Shin Guards} - CANDIDATES FOR RECOMMENDATION: {Soccer Jersey, Basketball Jersey, Football Jersey, Baseball Jersey, Tennis Shirt, Hockey Jersey, Soccer Ball, Basketball, Football, Baseball, Tennis Ball, Hocket Puck, Soccer Cleats, Basketball Shoes, Football Cleats, Baseball Cleats, Tennis Shoes, Hockey Helmet, Goalie Gloves, Basketball Arm Sleeve, Football Shoulder Pads, Baseball Cap, Tennis Racket, Hockey Skates, Soccer Goal Post, Basketball Hoop, Football Helmet, Baseball Bat, Hockey Stick, Soccer Cones, Basketball Shorts, Baseball Glove, Hockey Pads, Soccer Shorts} - RECOMMENDATION: 
RECOMMENDATION:  Basketball Arm Sleeve


In [ ]:
print(example_patient_2)
print("RECOMMENDATION: ", model_recommendation_2)

ITEMS PURCHASED: {Soccer Jersey, Soccer Goal Post, Soccer Cleats, Goalie Gloves} - CANDIDATES FOR RECOMMENDATION: {Basketball Jersey, Football Jersey, Baseball Jersey, Tennis Shirt, Hockey Jersey, Soccer Ball, Basketball, Football, Baseball, Tennis Ball, Hocket Puck, Basketball Shoes, Football Cleats, Baseball Cleats, Tennis Shoes, Hockey Helmet, Basketball Arm Sleeve, Football Shoulder Pads, Baseball Cap, Tennis Racket, Hockey Skates, Basketball Hoop, Football Helmet, Baseball Bat, Hockey Stick, Soccer Cones, Basketball Shorts, Baseball Glove, Hockey Pads, Soccer Shin Guards, Soccer Shorts} - RECOMMENDATION: 
RECOMMENDATION:  Soccer Ball


In [ ]:
print(example_patient_3)
print("RECOMMENDATION: ", model_recommendation_3)

ITEMS PURCHASED: {Basketball Jersey, Basketball, Basketball Arm Sleeve} - CANDIDATES FOR RECOMMENDATION: {Soccer Jersey, Football Jersey, Baseball Jersey, Tennis Shirt, Hockey Jersey, Soccer Ball, Football, Baseball, Tennis Ball, Hocket Puck, Soccer Cleats, Basketball Shoes, Football Cleats, Baseball Cleats, Tennis Shoes, Hockey Helmet, Goalie Gloves, Football Shoulder Pads, Baseball Cap, Tennis Racket, Hockey Skates, Soccer Goal Post, Basketball Hoop, Football Helmet, Baseball Bat, Hockey Stick, Soccer Cones, Basketball Shorts, Baseball Glove, Hockey Pads, Soccer Shin Guards, Soccer Shorts} - RECOMMENDATION: 
RECOMMENDATION:  Basketball Shoes
